In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-13 16:11:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2021-12-13 16:11:55 (10.2 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [5]:
# Read in data from S3 Buckets
dataset_1 = "amazon_reviews_us_Office_Products_v1_00.tsv.gz"
dataset_2 = "amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
url_1 = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + dataset_1
url_2 = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + dataset_2

from pyspark import SparkFiles
spark.sparkContext.addFile(url_1)
office_products_df = spark.read.csv(SparkFiles.get(dataset_1), sep="\t", header=True, inferSchema=True)

spark.sparkContext.addFile(url_2)
health_personal_care_df = spark.read.csv(SparkFiles.get(dataset_2), sep="\t", header=True, inferSchema=True)

In [6]:
# Show DataFrame
office_products_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [7]:
# Show DataFrame
health_personal_care_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [8]:
# Count the number of records (rows) in the dataset
print("Number of records in office_products_df: " + str(office_products_df.count()))
print("Number of records in health_personal_care_df: " + str(health_personal_care_df.count()))

Number of records in office_products_df: 2642434
Number of records in health_personal_care_df: 5331449


In [9]:
# Append dataframes
amazon_df = office_products_df.union(health_personal_care_df)

In [10]:
# Transform the dataset to fit the tables in the schema
from pyspark.sql.functions import to_date

# review_id_table
review_id_table = amazon_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-mm-dd').alias("review_date")])

# products
products = amazon_df.select(["product_id", "product_title"])
products = products.drop_duplicates(subset=['product_id'])

# customers
customers = amazon_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed('count(customer_id)','customer_count')
customers = customers.drop_duplicates(subset=['customer_id'])

# vine_table
vine_table = amazon_df[["review_id", "star_rating", "helpful_votes", "total_votes", "vine"]]

# Convert string to integer
from pyspark.sql.types import IntegerType
vine_table = vine_table.withColumn("star_rating", vine_table["star_rating"].cast(IntegerType()))

In [11]:
# Load the DataFrames that correspond to tables into an RDS instance
mode = "append"
jdbc_url = "jdbc:postgresql://<amazon-endpoint-address>:5432/<db-name>"
config = {
    "user" : "username", 
    "password" : "password",
    "driver" : "org.postgresql.Driver"
}

In [12]:
# Load review_id_table
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [13]:
# Load products
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [14]:
# Load customers
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [15]:
# Load vine_table
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)